In [1]:
library('ggplot2')
library(plyr)
library('dplyr')
library('tidyverse')
library('ggsci')
library('viridis')


Attache Paket: ‘dplyr’


Die folgenden Objekte sind maskiert von ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


Die folgenden Objekte sind maskiert von ‘package:stats’:

    filter, lag


Die folgenden Objekte sind maskiert von ‘package:base’:

    intersect, setdiff, setequal, union




ERROR: Error in library("tidyverse"): es gibt kein Paket namens ‘tidyverse’


In [2]:
list_paths <- function(filename){
  paths <- list.files(pattern=filename, recursive=TRUE)
  return(paths)
}

get_param_value <- function(param){
    return(as.double(str_split(param,"-")[[1]][2]))
    }

extract_parameters <- function(path){
    config <- str_split(path, "/")
    param <- lapply(str_split(config[[1]][length(str_split(path,"/")[[1]])-1], "_")[[1]],get_param_value)
    df <- read.csv(path, sep='\t')%>%
        add_column(b_cf=param[[2]][1],bc2c=param[[3]][1],cV=param[[4]][1],pi=param[[5]][1])}

In [3]:
paths <- list_paths("^logger_3.csv")

In [4]:
populations <- lapply(paths, extract_parameters)
populations <- do.call("rbind", populations)

ERROR: Error in str_split(path, "/"): konnte Funktion "str_split" nicht finden


In [6]:
inf_percent <- populations%>%mutate(percent_inf=100*celltype.infected.size/(celltype.target.size+celltype.infected.size))%>%
    group_by(time,b_cf,bc2c,cV,pi)%>%
    summarise(mean=mean(percent_inf),sd=sd(percent_inf))

inf_total <- populations%>%group_by(time,b_cf,bc2c,cV,pi)%>%
    summarise(infected_mean=mean(celltype.infected.size),
      infected_sd=sd(celltype.infected.size),
      target_mean=mean(celltype.target.size),
      target_sd=sd(celltype.target.size)
      )%>%gather(cell, value, -c(time,b_cf,bc2c,cV,pi))%>%
      separate(cell,c('cell', 'type'))%>%
      spread(type,value)

`summarise()` has grouped output by 'time', 'b_cf', 'bc2c', 'cV'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'time', 'b_cf', 'bc2c', 'cV'. You can override using the `.groups` argument.


In [7]:
head(inf_percent)

time,b_cf,bc2c,cV,pi,mean,sd
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,0.01,0.01,0.01,0.01,10,0
0,0.01,0.01,0.01,0.10,10,0
0,0.01,0.01,0.01,1.00,10,0
0,0.01,0.01,0.10,0.01,10,0
0,0.01,0.01,0.10,0.10,10,0
0,0.01,0.01,0.10,1.00,10,0


In [14]:
inf_percent <- populations%>%mutate(percent_inf=100*celltype.infected.size/(celltype.target.size+celltype.infected.size))%>%
    group_by(time,b_cf,bc2c,cV,pi)%>%
    summarise(mean=mean(percent_inf),sd=sd(percent_inf))%>%ungroup()

`summarise()` has grouped output by 'time', 'b_cf', 'bc2c', 'cV'. You can override using the `.groups` argument.


In [15]:
head(inf_percent)

time,b_cf,bc2c,cV,pi,mean,sd
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,0.01,0.01,0.01,0.01,10,0
0,0.01,0.01,0.01,0.10,10,0
0,0.01,0.01,0.01,1.00,10,0
0,0.01,0.01,0.10,0.01,10,0
0,0.01,0.01,0.10,0.10,10,0
0,0.01,0.01,0.10,1.00,10,0


In [22]:
ggplot(data=inf_percent, aes(x=time,y=mean, color=interaction(b_cf,bc2c,cV,pi,sep="-"), fill=interaction(b_cf,bc2c,cV,pi,sep="-"))) +
        geom_line(size=1) +
        geom_errorbar(mapping=aes(x=time, ymin=mean-sd, ymax=mean+sd, color=interaction(b_cf,bc2c,cV,pi,sep="-")), alpha=.3) + 
        labs(x="Time after infection [dpi]", y="Percentage of infected cells", fill="Scenario",color='Scenario') +
        facet_grid(b_cf~bc2c)

png 
  2

In [91]:
pdf(paste0(getwd(),"percent_infected.pdf"),width=8,height=3)
    ggplot(data=inf_percent, aes(x=time,y=mean, color=c(b_cf,bc2c,cV,pi), fill=c(b_cf,bc2c,cV,pi)) +
        geom_line(size=1) +
        geom_errorbar(mapping=aes(x=time, ymin=mean-sd, ymax=mean+sd, color=c(b_cf,bc2c,cV,pi)), alpha=.3) + 
        ggtitle("Infection dynamics") +
        labs(x="Time after infection [dpi]", y="Percentage of infected cells", fill="Scenario",color='Scenario') +
        theme_light() +
        scale_color_npg() +
        scale_fill_npg() +
        facet_grid(b_cf~bc2c, scales='free_y')
dev.off()
           
pdf(paste0(getwd(),"total_infected.pdf"),width=8,height=6)
ggplot(data=inf_total,aes(x=time,y=mean, color=c(b_cf,bc2c,cV,pi), fill=c(b_cf,bc2c,cV,pi)) +
  geom_line(size=1) +
  geom_errorbar(mapping=aes(x=time, ymin=mean-sd, ymax=mean+sd, color=c(b_cf,bc2c,cV,pi)), alpha=.3) +
  ggtitle("Infection dynamics") +
  labs(x="Time after infection [dpi]", y="Total cell numbers", fill="Scenario",color='Scenario') + theme_light() +
  scale_color_npg() +
  scale_fill_npg() +
  facet_grid(b_cf~bc2c,scales='free_y')
dev.off()

ERROR: Error in parse(text = x, srcfile = src): <text>:11:1: unerwartetes Symbol
10:         facet_grid(b_cf~bc2c, scales='free_y')
11: dev.off
    ^


In [84]:
populations

time,celltype.target.size,celltype.infected.size,b_cf,bc2c,cV,pi
<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
0,9,1,0.01,0.01,0.01,0.01
1,9,1,0.01,0.01,0.01,0.01
2,9,1,0.01,0.01,0.01,0.01
3,9,1,0.01,0.01,0.01,0.01
4,9,1,0.01,0.01,0.01,0.01
5,9,1,0.01,0.01,0.01,0.01
6,9,1,0.01,0.01,0.01,0.01
7,9,1,0.01,0.01,0.01,0.01
8,9,1,0.01,0.01,0.01,0.01
